In [118]:
# del load_filters

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings

from get_data_mapping import get_datamapping_pred_vs_last_year_demand, get_data_mapping_summary_product_sales, \
    get_data_mapping_accuracy_segmentation, get_data_mapping_confusion_matrix, get_data_mapping_sku_demand_page, \
    get_data_mapping_sku_demand_page_1
from pred_vs_last_year_demand import generate_prediction_vs_last_year_demand, generate_summary_product_sales, \
    get_accuracy_segmentation_schema, get_forecast_accuracy_schema, get_confusion_matrix_schema, \
    get_sku_demand_page_schema, get_confusion_matrix_schema_new, get_sku_demand_page_schema_1
from utility import make_input_data_continous, collect_data

warnings.filterwarnings('ignore')

output_folder_ = './DATA/RESULTS/'
mape_file_name = 'MAY_24_NEW'
sheet_name = "Schema_"

writer = pd.ExcelWriter(output_folder_ + mape_file_name + '.xlsx', engine='xlsxwriter')

final_forecast_path = 'DATA/MAY_24_FORECAST/combined_file_MAY_24.csv'
forecast_history_path = './DATA/FORECAST_HISTORY/forecast_history_dec_23.csv'
input_data_path = './DATA/INPUT_DATA/sales_groupby_31_07_24_final.csv'
final_forecast_data = pd.read_csv(final_forecast_path)
c = final_forecast_data['final_perdiction'].isna() == False
final_forecast_data = final_forecast_data[c]
forecast_history_data = pd.read_csv(forecast_history_path)
input_data = pd.read_csv(input_data_path)
print(final_forecast_data.shape)
print(forecast_history_data.shape)
print(input_data.shape)

(243438, 8)
(235734, 4)
(905252, 10)


In [2]:

import os

product_hierarchy_path = './DATA/Product_MASTER/'
product_hierarchy_files = os.listdir(product_hierarchy_path)
print(product_hierarchy_files)
col = ['Product', "NetWeight", 'H1_description', "H2_description",
       "H3_description", "H4_description", "H5_description", 'CrossPlantStatus', "LeadTime"
    , "ServiceLevel", 'MrpPurchaser', 'PdProductDescription']
sku_details = collect_data(product_hierarchy_path, product_hierarchy_files, col=col)
# master.rename(columns={'Product':"sku","CrossPlantStatus":"MSTAE"},inplace=True)
sku_details.rename(columns={'Product': "sku", 'H1_description': "H1", "H2_description": "H2"
    , "H3_description": 'H3', "H4_description": "H4", "H5_description": "H5"}, inplace=True)
sku_details['sku'] = sku_details['sku'].astype(int)

sku_details['ServiceLevel'] = sku_details['ServiceLevel'].astype(float)

stats_value_map = {
    .50: 0,
    .85: 1.04,
    .90: 1.28,
    .95: 1.65,
    .98: 2.05,
    .999: 3.09,
}
sku_details['ServiceLevel_value'] = sku_details['ServiceLevel'].map(stats_value_map)

c = sku_details['CrossPlantStatus'] == "AC"
active_list = sku_details[c]['sku'].unique()

print(input_data.head())
c = input_data['MATERIAL'].isin(active_list)
input_data = input_data[c]


print(final_forecast_data.head())
c = final_forecast_data['sku'].isin(active_list)
final_forecast_data = final_forecast_data[c]

sku_details.head()
# print(final_forecast_data['year_month'].head())
final_forecast_data['forecast_month'] = final_forecast_data['year_month']
# print(final_forecast_data['forecast_month'].head()+"-01")
print(pd.to_datetime(final_forecast_data['year_month'] + "-01"))
current_date = pd.to_datetime(final_forecast_data['forecast_month'] + "-01").min() - pd.DateOffset(months=1)

input_data['CHNL_NAME'] = np.where(input_data['CHNL_NAME'].isin(['Export', 'Re-Export']), 'Export_Re-Export',
                                   input_data['CHNL_NAME'])

input_data = input_data.groupby(['MATERIAL', 'year_month', 'CHNL_NAME'], as_index=False).sum()

input_data = pd.merge(input_data, sku_details[['sku', "H1"]],
                      left_on=['MATERIAL']
                      , right_on=['sku'], how='left')

['part-00000-47e15a93-a25c-4ce3-ac18-0e1b98d6d35a-c000.snappy.parquet', 'part-00001-47e15a93-a25c-4ce3-ac18-0e1b98d6d35a-c000.snappy.parquet', 'part-00002-47e15a93-a25c-4ce3-ac18-0e1b98d6d35a-c000.snappy.parquet', 'part-00003-47e15a93-a25c-4ce3-ac18-0e1b98d6d35a-c000.snappy.parquet']
./DATA/Product_MASTER/part-00000-47e15a93-a25c-4ce3-ac18-0e1b98d6d35a-c000.snappy.parquet
(6144, 12)
./DATA/Product_MASTER/part-00001-47e15a93-a25c-4ce3-ac18-0e1b98d6d35a-c000.snappy.parquet
(6144, 12)
./DATA/Product_MASTER/part-00002-47e15a93-a25c-4ce3-ac18-0e1b98d6d35a-c000.snappy.parquet
(6144, 12)
./DATA/Product_MASTER/part-00003-47e15a93-a25c-4ce3-ac18-0e1b98d6d35a-c000.snappy.parquet
(6531, 12)
   MATERIAL year_month CHNL_NAME  TOTAL_QTY_BASEUOM_SUM  \
0     101.0    2018-12    Retail                      1   
1     101.0    2019-01       B2B                      9   
2     101.0    2019-01    Retail                      9   
3     101.0    2019-02       B2B                      8   
4     101.0    2

In [3]:
input_data.drop(['sku'], axis=1, inplace=True)
print(input_data.head().T)
# c = input_data['H1'].isin(["Dairy"])
# input_data = input_data[c]
print(input_data['H1'].unique(),'Printing_DATA')
print(final_forecast_data.head(),'Printing_FF')

input_data_c = make_input_data_continous(input_data, current_date)
input_data_c['date'] = pd.to_datetime(input_data_c['year_month'] + "-01")

input_data_c.sort_values(['MATERIAL', 'H1', "CHNL_NAME", 'date'], inplace=True)

# Calculating Last Year Demand for SKU H1 & CHNL_NAME
input_data_c['last_year_demand'] = input_data_c.groupby(['MATERIAL', 'H1', 'CHNL_NAME'])['TOTAL_QTY_BASEUOM_SUM'].shift(12)
input_data_c['last_year_demand_non_contract'] = input_data_c.groupby(['MATERIAL', 'H1', 'CHNL_NAME'])['QTY_BASEUOM_SUM'].shift(12)
input_data_c['last_year_KG_SUM'] = input_data_c.groupby(['MATERIAL', 'H1', 'CHNL_NAME'])['KG_SUM'].shift(12)

final_forecast_data['year_month'] = final_forecast_data['forecast_month']

# c1=input_data['H1']=="Dairy"
# c2=input_data['CHNL_NAME']=="B2B"
# c3=input_data['year_month']=="2024-07"
# a=input_data[c1 & c2 & c3]['TOTAL_QTY_BASEUOM_SUM'].sum()
# print(a)

# ADDING UNIT COST START

input_d = input_data_c.copy()
c = input_d['TOTAL_QTY_BASEUOM_COUNT'] > 0
c1 = input_d['TOTAL_QTY_BASEUOM_SUM'] > 0
input_d = input_d[c & c1]
# input_d.shape
max_date_input = input_d.groupby(['MATERIAL', 'CHNL_NAME'], as_index=False)['date'].max()
max_date_input.rename(columns={"date": "max_date"}, inplace=True)
# max_date_input
input_d = pd.merge(input_d, max_date_input, on=['MATERIAL', 'CHNL_NAME'], how='left')
c = input_d['date'] == input_d['max_date']
input_d = input_d[c][['MATERIAL', 'CHNL_NAME', "TOTAL_SALES_SUM", 'TOTAL_QTY_BASEUOM_SUM']]

input_d["one_unit_cost"] = input_d['TOTAL_SALES_SUM'] / input_d['TOTAL_QTY_BASEUOM_SUM']
# input_d[['MATERIAL', 'CHNL_NAME', 'one_unit_cost']].to_clipboard(index=False)
input_data_c = pd.merge(input_data_c, input_d[['MATERIAL', 'CHNL_NAME', 'one_unit_cost']], on=['MATERIAL', 'CHNL_NAME'],
                        how='left')

# ADDING UNIT COST end

input_data_c.rename(columns={"MATERIAL": "sku", 'CHNL_NAME': "Channel"}, inplace=True)
# input_data_c.to_clipboard(index=False,sep=',')


                               0        1        2        3        4
MATERIAL                   101.0    101.0    101.0    101.0    101.0
year_month               2018-12  2019-01  2019-01  2019-02  2019-02
CHNL_NAME                 Retail      B2B   Retail      B2B   Retail
TOTAL_QTY_BASEUOM_SUM          1        9        9        8       10
TOTAL_QTY_BASEUOM_COUNT        1        6        8        3        8
TOTAL_SALES_SUM             14.3    121.5   125.81    108.0   138.65
QTY_BASEUOM_SUM              1.0      9.0      9.0      8.0     10.0
QTY_BASEUOM_COUNT            1.0      6.0      8.0      3.0      8.0
SALES_SUM                   14.3    121.5   125.81    108.0   138.65
KG_SUM                       5.0     45.0     45.0     40.0     50.0
H1                        Bakery   Bakery   Bakery   Bakery   Bakery
['Bakery' 'Chocolates' 'Fruits' 'Flour, Grain & Flakes'
 'Nuts, Seeds & Beans' 'Non Food' 'Beverage' 'Fat & Oil' 'Dairy'
 'Filling & Jam' 'Grocery' 'Cheese' 'PHD' 'Trade' '

In [4]:
final_file_schema_1 = pd.merge(final_forecast_data,
                               input_data_c[
                                   ['sku', 'year_month', 'H1', 'Channel','last_year_demand_non_contract', 'last_year_demand', 'one_unit_cost']],
                               on=['sku', 'H1', 'Channel', 'year_month'], how='left')

final_file_schema_1 = pd.merge(final_file_schema_1, sku_details[['sku', 'NetWeight']], on='sku', how='left')
final_file_schema_1['NetWeight'] = final_file_schema_1['NetWeight'].astype(float)

data_mapping_prediction_vs_last_year_demand = get_datamapping_pred_vs_last_year_demand(final_file_schema_1)

# Section to join revised sale
# For now attaching revised sales as 0 to all
final_file_schema_1['Revised_Sale'] = 0

# SCHEMA 1 PREDICTION VS LAST YEAR DEMAND
pred_vs_lyd_schema = generate_prediction_vs_last_year_demand(final_file_schema_1,
                                                             data_mapping_prediction_vs_last_year_demand)

pred_vs_lyd_schema.to_csv('./DATA/RESULTS/pred_vslast.csv', index=False, sep=',')

pred_vs_lyd_schema['prediction'] = np.ceil(pred_vs_lyd_schema['prediction'])

pred_vs_lyd_schema['last_year_demand'] = np.ceil(pred_vs_lyd_schema['last_year_demand'])

pred_vs_lyd_schema.to_excel(writer, sheet_name=sheet_name + "PRED_VS_LAST_YEAR_DEMAND", index=False)

# pred_vs_lyd_schema.to_clipboard(index=False,sep=',')

# SCHEMA PREDICTION VS LAST YEAR DEMAND - END

# SCHEMA PRODUCT SALES SUMMARY - START
c = input_data_c['date'] <= current_date
c2 = input_data_c['date'] > current_date - pd.DateOffset(months=6)
summary_product_sales_data = input_data_c[c & c2]

summary_product_sales_data = pd.merge(summary_product_sales_data, sku_details[['sku', 'NetWeight']], on=['sku'],
                                      how='left')
print(summary_product_sales_data.head().T)
summary_product_sales_data['NetWeight'] = summary_product_sales_data['NetWeight'].astype(float)

data_mapping_summary_product_sales = get_data_mapping_summary_product_sales(input_data_c)

# SCHEMA 2 SUMMARY PRODUCT SALES
product_sales_schema_data = generate_summary_product_sales(summary_product_sales_data,
                                                           data_mapping_summary_product_sales)
product_sales_schema_data['client_id'] = 1

product_sales_schema_data['last_updated_at'] = datetime.now().date()


                                                 0                    1  \
date                           2023-12-01 00:00:00  2024-01-01 00:00:00   
sku                                          101.0                101.0   
year_month                                 2023-12              2024-01   
Channel                                        B2B                  B2B   
TOTAL_QTY_BASEUOM_SUM                         13.0                 11.0   
TOTAL_QTY_BASEUOM_COUNT                        7.0                  7.0   
TOTAL_SALES_SUM                              175.5                148.5   
QTY_BASEUOM_SUM                               13.0                 11.0   
QTY_BASEUOM_COUNT                              7.0                  7.0   
SALES_SUM                                    175.5                148.5   
KG_SUM                                        65.0                 55.0   
H1                                          Bakery               Bakery   
last_year_demand         

In [5]:
product_sales_schema_data["Current Sales"] = product_sales_schema_data["Current Sales"]
product_sales_schema_data["Last Year Sales"] = product_sales_schema_data["Last Year Sales"]

product_sales_schema_data.rename(columns={
    "Current Sales": "current_sales",
    "Last Year Sales": "last_year_sales",
    "H1": "h1",
    "Channel": "channel",
    "Units_Base_UOM": "units_base_uom",
    "Value": "value",
}, inplace=True)

product_sales_schema_data['year_month'] = product_sales_schema_data['year_month'] + "-01"

product_sales_schema_data.to_excel(writer, sheet_name=sheet_name + "PRODUCT_SALES", index=False)

product_sales_schema_data.to_csv('./DATA/RESULTS/prod_summary.csv', index=False, sep=',')
# SCHEMA PRODUCT SALES SUMMARY - END

# SCHEMA ACCURACY SEGMENTATION & Summary: Forecast Accuracy START
sku_channel_history = forecast_history_data[['sku', 'Channel']].drop_duplicates()
sku_channel_history_data = pd.DataFrame()
for i in range(sku_channel_history.shape[0]):
    # print(i)
    sku_details_ = sku_channel_history.iloc[i, :]
    # print(sku_details_)
    d = pd.DataFrame()
    d['year_month'] = pd.Series(
        pd.date_range(current_date - pd.DateOffset(months=6), current_date, freq="M")) + pd.DateOffset(days=1)
    d['year_month'] = d['year_month'].astype(str).str[:-3]
    d['sku'] = sku_details_['sku']
    d['Channel'] = sku_details_['Channel']
    # display(d)
    sku_channel_history_data = pd.concat([sku_channel_history_data, d], axis=0)

# print(sku_channel_history_data.head())
# print(forecast_history_data.head())
forecast_history_data = pd.merge(sku_channel_history_data, forecast_history_data, on=['sku', 'year_month', 'Channel'],
                                 how='left')

In [26]:
# forecast_history_data

In [159]:
accuracy_segmentation = pd.merge(forecast_history_data,
                                 input_data_c[['sku', 'year_month', 'Channel', 'QTY_BASEUOM_SUM', 'SALES_SUM']],
                                 on=['sku', 'year_month', 'Channel'], how='left')
accuracy_segmentation

,year_month,sku,Channel,final_perdiction,QTY_BASEUOM_SUM,SALES_SUM
0,2023-12,1039,Export_Re-Export,6.620000e-24,0.0,0.0
1,2024-01,1039,Export_Re-Export,6.620000e-24,0.0,0.0
2,2024-02,1039,Export_Re-Export,6.620000e-24,0.0,0.0
3,2024-03,1039,Export_Re-Export,6.620000e-24,0.0,0.0
4,2024-04,1039,Export_Re-Export,6.620000e-24,0.0,0.0
...,...,...,...,...,...,...
235729,2024-01,200852,Ecomm,0.000000e+00,NaN,NaN
235730,2024-02,200852,Ecomm,0.000000e+00,NaN,NaN
235731,2024-03,200852,Ecomm,0.000000e+00,NaN,NaN
235732,2024-04,200852,Ecomm,0.000000e+00,NaN,NaN


In [268]:
def get_accuracy_segmentation_schema_1(acc_final_data, data_mapping_accuracy_segmentation):
    accuracy_segmentation_schema = pd.DataFrame()
    for i in range(data_mapping_accuracy_segmentation.shape[0]):
        filters = data_mapping_accuracy_segmentation.iloc[i, :]
        columns_ = ["Statistical Accuracy", "Total Accuracy"]
        rules_ = filters.to_frame().loc[columns_, :]
        rules_index = rules_.index
        rules_values = rules_.iloc[:, 0].values
        rules_values = [rules_index[i] for i, j in enumerate(rules_values) if int(j) != 0]
        rules_values = '_'.join(rules_values)
        print(rules_values)
        final_data = get_data_from_rules(acc_final_data, rules_values, 'segmentation_accuracy')
        for j in ['H1', "Channel"]:
            # final_data.to_clipboard(index=False)
            print(final_data.columns)
            print(j,filters[j],final_data)
            c = generate_condition(final_data, j, filters[j])
            print(c)
            final_data = final_data[c].reset_index(drop=True)
            if final_data.shape[0] == 0:
                continue
        if final_data.shape[0] == 0:
            continue
        # print(final_data['H1'].unique())
        # print(final_data['Channel'].unique())
        print(final_data.shape)
        # final_data.to_clipboard(index=False)
        # return None
        sales_sum = pd.pivot_table(final_data, values=['SALES_TOTAL'], aggfunc=['sum'], index=['year_month'],
                                   columns=["ACC_BUCKET"])
        sales_sum = pd.pivot_table(final_data, values=['SALES_TOTAL'], aggfunc=['sum'], index=['year_month'],
                                   columns=["ACC_BUCKET"])
        sales_sum_cols = ['_'.join(i) for i in sales_sum.columns]
        sales_sum.columns = sales_sum_cols
        sales_sum_t = sales_sum.T
        sales_sum = (sales_sum_t / sales_sum_t.sum()).T
        # sales_sum
        sku_count = pd.pivot_table(final_data, values=['sku'], aggfunc=['count'], index=['year_month'],
                                   columns=["ACC_BUCKET"])
        sku_cols = ['_'.join(i) for i in sku_count.columns]
        sku_count.columns = sku_cols
        sku_count_t = sku_count.T
        # sku_count = (sku_count_t / sku_count_t.sum()).T
        sku_count = sku_count_t.T
        # sku_count
        data_ = pd.concat([sales_sum, sku_count], axis=1).reset_index()
        data_['Channel'] = filters['Channel']
        data_['H1'] = filters['H1']
        data_[columns_] = 0
        data_[rules_values] = 1
        accuracy_segmentation_schema = pd.concat([accuracy_segmentation_schema, data_], axis=0)
    return accuracy_segmentation_schema.reset_index(drop=True)



In [161]:
from datetime import datetime

import numpy as np
import pandas as pd
from tqdm import tqdm

from schema_data_rules import get_data_from_rules, get_data_from_rules_1
from utility import generate_condition, load_filters

In [162]:
accuracy_segmentation['QTY_BASEUOM_SUM'].fillna(0, inplace=True)
accuracy_segmentation['SALES_SUM'].fillna(0, inplace=True)

accuracy_segmentation

,year_month,sku,Channel,final_perdiction,QTY_BASEUOM_SUM,SALES_SUM
0,2023-12,1039,Export_Re-Export,6.620000e-24,0.0,0.0
1,2024-01,1039,Export_Re-Export,6.620000e-24,0.0,0.0
2,2024-02,1039,Export_Re-Export,6.620000e-24,0.0,0.0
3,2024-03,1039,Export_Re-Export,6.620000e-24,0.0,0.0
4,2024-04,1039,Export_Re-Export,6.620000e-24,0.0,0.0
...,...,...,...,...,...,...
235729,2024-01,200852,Ecomm,0.000000e+00,0.0,0.0
235730,2024-02,200852,Ecomm,0.000000e+00,0.0,0.0
235731,2024-03,200852,Ecomm,0.000000e+00,0.0,0.0
235732,2024-04,200852,Ecomm,0.000000e+00,0.0,0.0


In [173]:
# Attachin
accuracy_segmentation['Revision_Sales'] = 2000

# acc_final_data = pd.merge(accuracy_segmentation, sku_details[['sku', 'H1']], on='sku', how='left')
acc_final_data=accuracy_segmentation.copy()
acc_final_data

,year_month,sku,Channel,final_perdiction,QTY_BASEUOM_SUM,SALES_SUM,Revision_Sales
0,2023-12,1039,Export_Re-Export,6.620000e-24,0.0,0.0,2000
1,2024-01,1039,Export_Re-Export,6.620000e-24,0.0,0.0,2000
2,2024-02,1039,Export_Re-Export,6.620000e-24,0.0,0.0,2000
3,2024-03,1039,Export_Re-Export,6.620000e-24,0.0,0.0,2000
4,2024-04,1039,Export_Re-Export,6.620000e-24,0.0,0.0,2000
...,...,...,...,...,...,...,...
235729,2024-01,200852,Ecomm,0.000000e+00,0.0,0.0,2000
235730,2024-02,200852,Ecomm,0.000000e+00,0.0,0.0,2000
235731,2024-03,200852,Ecomm,0.000000e+00,0.0,0.0,2000
235732,2024-04,200852,Ecomm,0.000000e+00,0.0,0.0,2000


In [174]:
acc_final_data.rename(columns={"QTY_BASEUOM_SUM": "actual_demand"}, inplace=True)

acc_final_data['mape'] = np.abs(acc_final_data['actual_demand'] - acc_final_data['final_perdiction']) / acc_final_data[
    'actual_demand']
c1 = acc_final_data['mape'] <= .25
c2 = acc_final_data['mape'] > 0.5
c3 = (acc_final_data['mape'] > 0.25) & (acc_final_data['mape'] <= 0.5)
acc_final_data["ACC_BUCKET"] = np.select([c1, c2, c3], ["HIGH", "LOW", 'MEDIUM'], "NA")
acc_final_data["ACC_BUCKET"] = np.where(np.isinf(acc_final_data['mape']), 'NA', acc_final_data['ACC_BUCKET'])
print(acc_final_data['mape'].isna().sum())
acc_final_data["mape"] = np.where(np.isinf(acc_final_data['mape']), 'NA', acc_final_data['mape'])
acc_final_data["mape"] = np.where(acc_final_data['mape'].isna(), 'NA', acc_final_data['mape'])
acc_final_data

41884


,year_month,sku,Channel,final_perdiction,actual_demand,SALES_SUM,Revision_Sales,mape,ACC_BUCKET
0,2023-12,1039,Export_Re-Export,6.620000e-24,0.0,0.0,2000,NA,NA
1,2024-01,1039,Export_Re-Export,6.620000e-24,0.0,0.0,2000,NA,NA
2,2024-02,1039,Export_Re-Export,6.620000e-24,0.0,0.0,2000,NA,NA
3,2024-03,1039,Export_Re-Export,6.620000e-24,0.0,0.0,2000,NA,NA
4,2024-04,1039,Export_Re-Export,6.620000e-24,0.0,0.0,2000,NA,NA
...,...,...,...,...,...,...,...,...,...
235729,2024-01,200852,Ecomm,0.000000e+00,0.0,0.0,2000,nan,NA
235730,2024-02,200852,Ecomm,0.000000e+00,0.0,0.0,2000,nan,NA
235731,2024-03,200852,Ecomm,0.000000e+00,0.0,0.0,2000,nan,NA
235732,2024-04,200852,Ecomm,0.000000e+00,0.0,0.0,2000,nan,NA


In [175]:
input_data_c

,date,sku,year_month,Channel,TOTAL_QTY_BASEUOM_SUM,TOTAL_QTY_BASEUOM_COUNT,TOTAL_SALES_SUM,QTY_BASEUOM_SUM,QTY_BASEUOM_COUNT,SALES_SUM,KG_SUM,H1,last_year_demand,last_year_demand_non_contract,last_year_KG_SUM,one_unit_cost
0,2018-12-01,101.0,2018-12,B2B,NaN,NaN,NaN,0.0,NaN,0.0,NaN,Bakery,NaN,NaN,NaN,13.50
1,2019-01-01,101.0,2019-01,B2B,9.0,6.0,121.5,9.0,6.0,121.5,45.0,Bakery,NaN,NaN,NaN,13.50
2,2019-02-01,101.0,2019-02,B2B,8.0,3.0,108.0,8.0,3.0,108.0,40.0,Bakery,NaN,NaN,NaN,13.50
3,2019-03-01,101.0,2019-03,B2B,20.0,9.0,264.0,20.0,9.0,264.0,100.0,Bakery,NaN,NaN,NaN,13.50
4,2019-04-01,101.0,2019-04,B2B,5.0,5.0,67.5,5.0,5.0,67.5,25.0,Bakery,NaN,NaN,NaN,13.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1332137,2024-07-01,670176.0,2024-07,Export_Re-Export,NaN,NaN,NaN,0.0,NaN,0.0,NaN,Non Food,NaN,0.0,NaN,0.12
1332138,2024-08-01,670176.0,2024-08,Export_Re-Export,NaN,NaN,NaN,0.0,NaN,0.0,NaN,Non Food,NaN,0.0,NaN,0.12
1332139,2024-09-01,670176.0,2024-09,Export_Re-Export,NaN,NaN,NaN,0.0,NaN,0.0,NaN,Non Food,NaN,0.0,NaN,0.12
1332140,2024-10-01,670176.0,2024-10,Export_Re-Export,NaN,NaN,NaN,0.0,NaN,0.0,NaN,Non Food,NaN,0.0,NaN,0.12


In [176]:
sku_details

,sku,NetWeight,H1,H2,H3,H4,H5,CrossPlantStatus,LeadTime,ServiceLevel,MrpPurchaser,PdProductDescription,ServiceLevel_value
0,101,5.000,Bakery,Functional,Leavening,Others,None,AC,1,0.95,LESTER SEOW,AMMONIUM BICARBONATE REDMAN 5KG,1.65
1,102,25.000,Bakery,Functional,Leavening,Others,None,AC,2,0.95,LESTER SEOW,AMMONIUM BICARBONATE 25KG,1.65
2,105,18.000,Bakery,Functional,Leavening,Baking Powder,None,AC,1,0.95,JASYLN,BAKING POWDER REDMAN 18KG,1.65
3,112,5.000,Bakery,Functional,Leavening,Others,None,AC,1,0.95,LESTER SEOW,SODIUM BICARBONATE REDMAN 5KG,1.65
4,116,0.250,Bakery,Functional,Stabilizer,Others,None,AC,1,0.95,LESTER SEOW,SPONGE CAKE GEL REDMAN 250G,1.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24958,680070,1.000,Non Food,Consumables,Packaging,Others,None,AC,2,0.95,SENG LIAN,"BAG SINGLET TRANSLUCENT 21X23"" 30PC",1.65
24959,680081,1.000,Non Food,Consumables,Packaging,Flexible Packaging,None,AC,3,0.95,SENG LIAN,PE FOAM MESH SLEEVE 100MM(W)X200MM(L),1.65
24960,680100,0.200,Non Food,Consumables,Apparels & Gloves,Apparels,None,AC,None,NaN,None,T-SHIRT S.SLEEVE POLO UNISEX-NAVY 3XL,NaN
24961,680101,5.000,PHD,Non Food,Chemical,Others,None,AC,None,NaN,None,DETERGENT GENIE MULTIPURPOSE 5L,NaN


In [177]:
acc_final_data=pd.merge(acc_final_data,sku_details[['sku','H1']],how='left',on=['sku'])
acc_final_data

,year_month,sku,Channel,final_perdiction,actual_demand,SALES_SUM,Revision_Sales,mape,ACC_BUCKET,H1
0,2023-12,1039,Export_Re-Export,6.620000e-24,0.0,0.0,2000,NA,NA,Dairy
1,2024-01,1039,Export_Re-Export,6.620000e-24,0.0,0.0,2000,NA,NA,Dairy
2,2024-02,1039,Export_Re-Export,6.620000e-24,0.0,0.0,2000,NA,NA,Dairy
3,2024-03,1039,Export_Re-Export,6.620000e-24,0.0,0.0,2000,NA,NA,Dairy
4,2024-04,1039,Export_Re-Export,6.620000e-24,0.0,0.0,2000,NA,NA,Dairy
...,...,...,...,...,...,...,...,...,...,...
235729,2024-01,200852,Ecomm,0.000000e+00,0.0,0.0,2000,nan,NA,Seafood
235730,2024-02,200852,Ecomm,0.000000e+00,0.0,0.0,2000,nan,NA,Seafood
235731,2024-03,200852,Ecomm,0.000000e+00,0.0,0.0,2000,nan,NA,Seafood
235732,2024-04,200852,Ecomm,0.000000e+00,0.0,0.0,2000,nan,NA,Seafood


In [269]:
data_mapping_accuracy_segmentation = get_data_mapping_accuracy_segmentation(input_data_c)
# SCHEMA 3 ACCURACY SEGMENTATION
acc_final_data_schema = get_accuracy_segmentation_schema_1(acc_final_data, data_mapping_accuracy_segmentation)

acc_final_data_schema["client_id"] = 1
acc_final_data_schema['last_updated_at'] = datetime.now().date()
acc_final_data_schema

Statistical Accuracy
Index(['year_month', 'sku', 'Channel', 'final_perdiction', 'actual_demand',
       'SALES_SUM', 'Revision_Sales', 'mape', 'ACC_BUCKET', 'H1',
       'SALES_TOTAL'],
      dtype='object')
H1 Bakery        year_month     sku           Channel  final_perdiction  actual_demand  \
0         2023-12    1039  Export_Re-Export      6.620000e-24            0.0   
1         2024-01    1039  Export_Re-Export      6.620000e-24            0.0   
2         2024-02    1039  Export_Re-Export      6.620000e-24            0.0   
3         2024-03    1039  Export_Re-Export      6.620000e-24            0.0   
4         2024-04    1039  Export_Re-Export      6.620000e-24            0.0   
...           ...     ...               ...               ...            ...   
235729    2024-01  200852             Ecomm      0.000000e+00            0.0   
235730    2024-02  200852             Ecomm      0.000000e+00            0.0   
235731    2024-03  200852             Ecomm      0.000000e+00 

TypeError: 'NoneType' object does not support item assignment

In [270]:
dd=pd.read_clipboard()

In [271]:
dd

,year_month,sku,Channel,final_perdiction,actual_demand,SALES_SUM,Revision_Sales,mape,ACC_BUCKET,H1,SALES_TOTAL
0,2023-12,105,B2B,1.528591e+02,136.0,7276.0,2000,0.123964,HIGH,Bakery,7276.0
1,2024-01,105,B2B,1.314232e+02,125.0,6591.0,2000,0.051385,HIGH,Bakery,6591.0
2,2024-02,105,B2B,1.392068e+02,99.0,5306.0,2000,0.406130,MEDIUM,Bakery,5306.0
3,2024-03,105,B2B,1.506389e+02,184.0,9820.0,2000,0.181310,HIGH,Bakery,9820.0
4,2024-04,105,B2B,1.276851e+02,101.0,5337.0,2000,0.264209,MEDIUM,Bakery,5337.0
...,...,...,...,...,...,...,...,...,...,...,...
13633,2024-01,400182,B2B,3.140000e-12,0.0,0.0,2000,NaN,NaN,Bakery,0.0
13634,2024-02,400182,B2B,3.140000e-12,0.0,0.0,2000,NaN,NaN,Bakery,0.0
13635,2024-03,400182,B2B,3.140000e-12,0.0,0.0,2000,NaN,NaN,Bakery,0.0
13636,2024-04,400182,B2B,3.140000e-12,0.0,0.0,2000,NaN,NaN,Bakery,0.0


In [187]:
rename_col_for_acc_seg = {
    "sum_SALES_TOTAL_HIGH": "high_percentage_value",
    "sum_SALES_TOTAL_MEDIUM": "medium_percentage_value",
    "sum_SALES_TOTAL_LOW": "low_percentage_value",
    "sum_SALES_TOTAL_NA": "others_percentage_value",
    "count_sku_HIGH": "high_sku_count",
    "count_sku_MEDIUM": "medium_sku_count",
    "count_sku_LOW": "low_sku_count",
    "count_sku_NA": "others_sku_count",
    "H1": "h1",
    "Channel": "channel",
    "Statistical Accuracy": "statistical_accuracy",
    "Total Accuracy": "total_accuracy",
}

acc_final_data_schema.rename(columns=rename_col_for_acc_seg, inplace=True)

In [188]:
columns = ['year_month', 'high_percentage_value', 'low_percentage_value',
           'medium_percentage_value', 'others_percentage_value', 'high_sku_count',
           'low_sku_count', 'medium_sku_count', 'others_sku_count', 'channel',
           'h1', 'statistical_accuracy', 'total_accuracy', 'client_id',
           'last_updated_at']
actual_columns = set(columns)

columns_present = set(acc_final_data_schema.columns)

for i in actual_columns - columns_present:
    acc_final_data_schema[i] = np.nan

acc_final_data_schema['year_month'] = pd.to_datetime(acc_final_data_schema['year_month'] + "-01")
acc_final_data_schema

,year_month,high_percentage_value,low_percentage_value,medium_percentage_value,others_percentage_value,high_sku_count,low_sku_count,medium_sku_count,others_sku_count,channel,h1,statistical_accuracy,total_accuracy,client_id,last_updated_at
0,2023-12-01,0.615452,0.179999,0.204549,0.000000,219.0,353.0,155.0,1546.0,B2B,Bakery,1,0,1,2024-08-19
1,2024-01-01,0.597503,0.119305,0.283191,0.000000,209.0,322.0,159.0,1583.0,B2B,Bakery,1,0,1,2024-08-19
2,2024-02-01,0.555682,0.191314,0.253004,0.000000,176.0,326.0,132.0,1639.0,B2B,Bakery,1,0,1,2024-08-19
3,2024-03-01,0.590536,0.160229,0.249234,0.000000,194.0,292.0,156.0,1631.0,B2B,Bakery,1,0,1,2024-08-19
4,2024-04-01,0.580182,0.174850,0.244967,0.000000,176.0,327.0,146.0,1624.0,B2B,Bakery,1,0,1,2024-08-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,2024-01-01,0.137941,0.140274,0.082019,0.639766,3094.0,4927.0,2358.0,28910.0,ALL,ALL,0,1,1,2024-08-19
956,2024-02-01,0.113188,0.135225,0.076376,0.675211,2767.0,4725.0,2092.0,29705.0,ALL,ALL,0,1,1,2024-08-19
957,2024-03-01,0.114393,0.139053,0.088855,0.657698,2802.0,4830.0,2352.0,29305.0,ALL,ALL,0,1,1,2024-08-19
958,2024-04-01,0.125116,0.130135,0.078376,0.666373,2936.0,4659.0,2262.0,29432.0,ALL,ALL,0,1,1,2024-08-19


In [189]:
acc_final_data_schema.to_clipboard(index=False)

In [171]:
# SCHEMA 4 FORECAST ACCURACY
acc_final_data.isna().sum()
forecast_accuracy_schema = get_forecast_accuracy_schema(acc_final_data, data_mapping_accuracy_segmentation)
forecast_accuracy_schema.rename(columns={"Predicted": "predicted"}, inplace=True)

Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total A

In [49]:
forecast_accuracy_schema['year_month'] = pd.to_datetime(forecast_accuracy_schema['year_month'] + '-01')

forecast_accuracy_schema['client_id'] = 1
forecast_accuracy_schema['last_updated_at'] = datetime.now().date()
print("Acc forecast accuracy schema")
forecast_accuracy_schema.to_excel(writer, sheet_name=sheet_name + "FORECAST_ACC", index=False)
# SCHEMA ACCURACY SEGMENTATION & Summary: Forecast Accuracy END

Acc forecast accuracy schema


In [50]:
acc_final_data['SALES_SUM'].fillna(0, inplace=True)
acc_final_data['actual_demand'].fillna(0, inplace=True)

In [51]:
data_mapping_confusion_matrix = get_data_mapping_confusion_matrix(input_data_c)

In [195]:
def collect_data(path, col=None):
    files=os.listdir(master_path)
    d = pd.DataFrame()
    for i in files:
        file_path = path + i
        print(file_path)
        data = pd.read_parquet(file_path, columns=col)
        print(data.shape)
        d = pd.concat([d, data], axis=0)
    return d.reset_index(drop=True)


In [362]:
master_path='../DATA_MILAN/S3BUCKER/newProductehei_1/'
sku_master=collect_data(master_path)
sku_master

../DATA_MILAN/S3BUCKER/newProductehei_1/part-00000-47e15a93-a25c-4ce3-ac18-0e1b98d6d35a-c000.snappy.parquet
(6144, 28)
../DATA_MILAN/S3BUCKER/newProductehei_1/part-00001-47e15a93-a25c-4ce3-ac18-0e1b98d6d35a-c000.snappy.parquet
(6144, 28)
../DATA_MILAN/S3BUCKER/newProductehei_1/part-00002-47e15a93-a25c-4ce3-ac18-0e1b98d6d35a-c000.snappy.parquet
(6144, 28)
../DATA_MILAN/S3BUCKER/newProductehei_1/part-00003-47e15a93-a25c-4ce3-ac18-0e1b98d6d35a-c000.snappy.parquet
(6531, 28)


,Product,ProductHierarchy,Purchaser,ServiceLevel,MrpController,MrpPurchaser,GoodReceiptTime,LeadTime,BaseUnit,CreatedByUser,...,H4,H5,H1_description,H2_description,H3_description,H4_description,H5_description,PdProductDescription,Language,ClientId
0,101,533AGC5,LESTER SEOW,0.95,PROD HQ POWDER MIX,LESTER SEOW,2,1,BAG,HCC_DATA,...,533AGC5,None,Bakery,Functional,Leavening,Others,None,AMMONIUM BICARBONATE REDMAN 5KG,EN,1
1,102,533AGC5,LESTER SEOW,0.95,Local RS,LESTER SEOW,2,2,BAG,HCC_DATA,...,533AGC5,None,Bakery,Functional,Leavening,Others,None,AMMONIUM BICARBONATE 25KG,EN,1
2,105,533AG17,JASYLN,0.95,PROD HQ POWDER MIX,JASYLN,2,1,TIN,HCC_DATA,...,533AG17,None,Bakery,Functional,Leavening,Baking Powder,None,BAKING POWDER REDMAN 18KG,EN,1
3,112,533AGC5,LESTER SEOW,0.95,PROD HQ POWDER MIX,LESTER SEOW,2,1,BAG,HCC_DATA,...,533AGC5,None,Bakery,Functional,Leavening,Others,None,SODIUM BICARBONATE REDMAN 5KG,EN,1
4,116,533COC5,LESTER SEOW,0.95,PROD HQ MANUAL PAK,LESTER SEOW,2,1,BT,HCC_DATA,...,533COC5,None,Bakery,Functional,Stabilizer,Others,None,SPONGE CAKE GEL REDMAN 250G,EN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24958,680070,J15B3C5,SENG LIAN,0.95,Choumun,SENG LIAN,2,2,PAC,CHOUMUN,...,J15B3C5,None,Non Food,Consumables,Packaging,Others,None,"BAG SINGLET TRANSLUCENT 21X23"" 30PC",EN,1
24959,680081,J15B398,SENG LIAN,0.95,Choumun,SENG LIAN,2,3,BAG,CHOUMUN,...,J15B398,None,Non Food,Consumables,Packaging,Flexible Packaging,None,PE FOAM MESH SLEEVE 100MM(W)X200MM(L),EN,1
24960,680100,J150708,None,None,None,None,None,None,PC,FI_AUTO,...,J150708,None,Non Food,Consumables,Apparels & Gloves,Apparels,None,T-SHIRT S.SLEEVE POLO UNISEX-NAVY 3XL,EN,1
24961,680101,G4538C5,None,None,None,None,None,None,BT,FI_AUTO,...,G4538C5,None,PHD,Non Food,Chemical,Others,None,DETERGENT GENIE MULTIPURPOSE 5L,EN,1


In [199]:
c=sku_master['H1_description']=="Fruits"
c1=sku_master['CrossPlantStatus']=="AC"
sku_master[c & c1]

,Product,ProductHierarchy,Purchaser,ServiceLevel,MrpController,MrpPurchaser,GoodReceiptTime,LeadTime,BaseUnit,CreatedByUser,...,H4,H5,H1_description,H2_description,H3_description,H4_description,H5_description,PdProductDescription,Language,ClientId
22,231,D2149AR,LESTER SEOW,0.95,PROD HQ MANUAL PAK,LESTER SEOW,2,1,PAC,HCC_DATA,...,D2149AR,None,Fruits,Fruits Desiccated,Coconut,High Fat,None,COCONUT DESICCATED REDMAN 250G,EN,1
31,294,D2149AR,LESTER SEOW,0.95,PROD HQ POWDER MIX,LESTER SEOW,2,1,TIN,HCC_DATA,...,D2149AR,None,Fruits,Fruits Desiccated,Coconut,High Fat,None,COCONUT DESICCATED REDMAN 6KG,EN,1
137,703,D0637CZ,LESTER SEOW,0.95,PROD HQ MANUAL PAK,LESTER SEOW,2,1,PAC,HCC_DATA,...,D0637CZ,None,Fruits,Fruits Candied,Chelory,Red,None,CHELORY DICED RED REDMAN 250G,EN,1
138,705,D0637CZ,EVITA CHANG,0.95,Joyce,EVITA CHANG,2,50,CAR,HCC_DATA,...,D0637CZ,None,Fruits,Fruits Candied,Chelory,Red,None,CHELORY DICED RED/REDMAN 15KG,EN,1
139,707,D06ASC5,EVITA CHANG,0.95,PROD HQ MANUAL PAK,EVITA CHANG,2,1,PAC,HCC_DATA,...,D06ASC5,None,Fruits,Fruits Candied,Mixed Citrus Peel,Others,None,DRIED ORANGE PEEL CUBE 6X6MM REDMAN 250G,EN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23751,202523,D06ASC5,EVITA CHANG,0.95,PROD HQ MANUAL PAK,EVITA CHANG,2,2,PAC,FI_AUTO,...,D06ASC5,None,Fruits,Fruits Candied,Mixed Citrus Peel,Others,None,DRIED ORANGE PEEL STRIP (4.5-7CM) 1KG,EN,1
23752,202524,D06ASC5,EVITA CHANG,0.95,PROD HQ MANUAL PAK,EVITA CHANG,2,2,PAC,FI_AUTO,...,D06ASC5,None,Fruits,Fruits Candied,Mixed Citrus Peel,Others,None,DRIED ORANGE PEEL STRIP (4.5-7CM) 250G,EN,1
23994,300170,D06ASC5,EVITA CHANG,0.95,PROD HQ CLRS & FLV,EVITA CHANG,2,2,BAG,FI_AUTO,...,D06ASC5,None,Fruits,Fruits Candied,Mixed Citrus Peel,Others,None,MIXED CITRUS PEEL 6X6MM REDMAN 20KG-SFG,EN,1
24045,400129,D22AVEU,EVITA CHANG,0.95,NULL,EVITA CHANG,2,3,CAR,JOYCE.TAY,...,D22AVEU,None,Fruits,Fruits Dried,Natural,Whole,None,DRIED BLACK RAISINS/CALIFORNIA 13.62KG,EN,1


In [144]:
data_=pd.merge(forecast_history_data,sku_details[['sku','H1',]],on=['sku'],how='left')
data_

,year_month,sku,Channel,final_perdiction,H1
0,2023-12,1039,Export_Re-Export,6.620000e-24,Dairy
1,2024-01,1039,Export_Re-Export,6.620000e-24,Dairy
2,2024-02,1039,Export_Re-Export,6.620000e-24,Dairy
3,2024-03,1039,Export_Re-Export,6.620000e-24,Dairy
4,2024-04,1039,Export_Re-Export,6.620000e-24,Dairy
...,...,...,...,...,...
235729,2024-01,200852,Ecomm,0.000000e+00,Seafood
235730,2024-02,200852,Ecomm,0.000000e+00,Seafood
235731,2024-03,200852,Ecomm,0.000000e+00,Seafood
235732,2024-04,200852,Ecomm,0.000000e+00,Seafood


In [150]:
# input_data_c

In [151]:
cc=['sku','year_month','Channel','one_unit_cost','QTY_BASEUOM_SUM']
data_=pd.merge(data_,input_data_c[cc],on=['sku','Channel','year_month'],how='left')

In [152]:
data_['actual_sales']=data_['one_unit_cost']*data_['QTY_BASEUOM_SUM']
data_['forecast_sales']=data_['one_unit_cost']*data_['final_perdiction']

In [153]:
data_

,year_month,sku,Channel,final_perdiction,H1,one_unit_cost,QTY_BASEUOM_SUM,actual_sales,forecast_sales
0,2023-12,1039,Export_Re-Export,6.620000e-24,Dairy,240.0,0.0,0.0,1.588800e-21
1,2024-01,1039,Export_Re-Export,6.620000e-24,Dairy,240.0,0.0,0.0,1.588800e-21
2,2024-02,1039,Export_Re-Export,6.620000e-24,Dairy,240.0,0.0,0.0,1.588800e-21
3,2024-03,1039,Export_Re-Export,6.620000e-24,Dairy,240.0,0.0,0.0,1.588800e-21
4,2024-04,1039,Export_Re-Export,6.620000e-24,Dairy,240.0,0.0,0.0,1.588800e-21
...,...,...,...,...,...,...,...,...,...
235729,2024-01,200852,Ecomm,0.000000e+00,Seafood,NaN,NaN,NaN,NaN
235730,2024-02,200852,Ecomm,0.000000e+00,Seafood,NaN,NaN,NaN,NaN
235731,2024-03,200852,Ecomm,0.000000e+00,Seafood,NaN,NaN,NaN,NaN
235732,2024-04,200852,Ecomm,0.000000e+00,Seafood,NaN,NaN,NaN,NaN


In [158]:
c1=data_['H1']=="Bakery"
c2=data_['Channel']=="B2B"
c3=data_['year_month']=="2023-12"
c4=data_['actual_sales']<10000
c5=data_['forecast_sales']<10000
data_[c1 & c2 & c3 & c4 & c5].shape

(1341, 9)

In [123]:
c1=history_data['H1']=="Bakery"
c2=history_data['Channel']=="B2B"
c3=history_data['year_month']=="2023-12"
c4=history_data['SGD']>10000
c5=history_data['SGD']<100000
history_data[c1 & c2 & c3 & c4 & c5]

KeyError: 'H1'

In [113]:
acc_final_data["actual_sales"] = acc_final_data['actual_demand'] * acc_final_data['one_unit_cost']

In [114]:
acc_final_data

,year_month,sku,Channel,final_perdiction,H1,actual_demand,SALES_SUM,Revision_Sales,mape,ACC_BUCKET,one_unit_cost,actual_sales
0,2023-12,1039,Export_Re-Export,6.620000e-24,Dairy,0.0,0.0,0,NA,NA,240.0,0.0
1,2024-01,1039,Export_Re-Export,6.620000e-24,Dairy,0.0,0.0,0,NA,NA,240.0,0.0
2,2024-02,1039,Export_Re-Export,6.620000e-24,Dairy,0.0,0.0,0,NA,NA,240.0,0.0
3,2024-03,1039,Export_Re-Export,6.620000e-24,Dairy,0.0,0.0,0,NA,NA,240.0,0.0
4,2024-04,1039,Export_Re-Export,6.620000e-24,Dairy,0.0,0.0,0,NA,NA,240.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
235729,2024-01,200852,Ecomm,0.000000e+00,Seafood,0.0,0.0,0,nan,NA,NaN,NaN
235730,2024-02,200852,Ecomm,0.000000e+00,Seafood,0.0,0.0,0,nan,NA,NaN,NaN
235731,2024-03,200852,Ecomm,0.000000e+00,Seafood,0.0,0.0,0,nan,NA,NaN,NaN
235732,2024-04,200852,Ecomm,0.000000e+00,Seafood,0.0,0.0,0,nan,NA,NaN,NaN


In [105]:
# SCHEMA 5 CONFUSION MATRIX
confusion_matrix_schema = get_confusion_matrix_schema_(acc_final_data, data_mapping_confusion_matrix)
# confusion_matrix_schema = get_confusion_matrix_schema_new(acc_final_data, data_mapping_confusion_matrix, current_date)
confusion_matrix_schema['client_id'] = 1
confusion_matrix_schema['last_updated_at'] = datetime.now().date()
confusion_matrix_schema

Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistical
Total
Statistica

,Actual Sales,Predicted Sales,No of SKUS,year_month,H1,Channel,Total,Statistical,client_id,last_updated_at
0,< SGD 10000k,< SGD 10000k,1344,2023-12,Bakery,B2B,0,1,1,2024-08-18
1,SGD 10000K- 100000K,< SGD 10000k,0,2023-12,Bakery,B2B,0,1,1,2024-08-18
2,< SGD 10000k,SGD 10000K- 100000K,17,2023-12,Bakery,B2B,0,1,1,2024-08-18
3,SGD 10000K- 100000K,SGD 10000K- 100000K,1,2023-12,Bakery,B2B,0,1,1,2024-08-18
4,< SGD 10000k,0,911,2023-12,Bakery,B2B,0,1,1,2024-08-18
...,...,...,...,...,...,...,...,...,...,...
43,SGD 10000K- 100000K,SGD 10000K- 100000K,13,2024-05,ALL,ALL,1,0,1,2024-08-18
44,< SGD 10000k,0,16869,2024-05,ALL,ALL,1,0,1,2024-08-18
45,SGD 10000K- 100000K,0,0,2024-05,ALL,ALL,1,0,1,2024-08-18
46,< SGD 10000k,SGD 100000K- 200000K,0,2024-05,ALL,ALL,1,0,1,2024-08-18


In [102]:
confusion_matrix_schema
# .to_clipboard(index=False)

,Actual Sales,Predicted Sales,No of SKUS,year_month,H1,Channel,Total,Statistical,client_id,last_updated_at
0,SGD 100K- 200K,> SGD 200K,43,2023-12,Bakery,B2B,0,1,1,2024-08-18
1,SGD 10K- 100K,> SGD 200K,153,2023-12,Bakery,B2B,0,1,1,2024-08-18
2,> SGD 200K,> SGD 200K,96,2023-12,Bakery,B2B,0,1,1,2024-08-18
3,< SGD 10k,> SGD 200K,66,2023-12,Bakery,B2B,0,1,1,2024-08-18
4,SGD 100K- 200K,SGD 100K- 200K,4,2023-12,Bakery,B2B,0,1,1,2024-08-18
...,...,...,...,...,...,...,...,...,...,...
115,> SGD 200K,SGD 100K- 200K,24,2024-05,ALL,ALL,1,0,1,2024-08-18
116,< SGD 10k,0,16869,2024-05,ALL,ALL,1,0,1,2024-08-18
117,SGD 10K- 100K,0,0,2024-05,ALL,ALL,1,0,1,2024-08-18
118,SGD 100K- 200K,0,0,2024-05,ALL,ALL,1,0,1,2024-08-18


In [98]:
def get_confusion_matrix_schema_1(acc_final_data, data_mapping_accuracy_segmentation):
    confusion_matrix_schema = pd.DataFrame()
    for i in range(data_mapping_accuracy_segmentation.shape[0]):
        filters = data_mapping_accuracy_segmentation.iloc[i, :]
        columns_ = ["Statistical", "Total"]
        rules_ = filters.to_frame().loc[columns_, :]
        rules_index = rules_.index
        rules_values = rules_.iloc[:, 0].values
        rules_values = [rules_index[i] for i, j in enumerate(rules_values) if int(j) != 0]
        rules_values = '_'.join(rules_values)
        print(rules_values)
        final_data = get_data_from_rules(acc_final_data, rules_values, 'confusion_matrix')
        # display(final_data)
        limits = [100, 100, 200]
        cols = "Predicted"
        final_data["FINAL_PRED_FLAG"] = load_filters_(limits, cols, final_data)
        cols = "actual_sales"
        final_data["ACTUAL_DEMAND_FLAG"] = load_filters_(limits, cols, final_data)
        for j in ['H1', "Channel"]:
            # print(j,filters[j])
            c = generate_condition(final_data, j, filters[j])
            final_data = final_data[c].reset_index(drop=True)
        if final_data.shape[0] == 0:
            continue
        data_collect = {
            "Actual Sales": [],
            "Predicted Sales": [],
            "No of SKUS": [],
        }
        # print(final_data['ACTUAL_DEMAND_FLAG'].unique())
        # v=final_data['ACTUAL_DEMAND_FLAG']=="0"
        # display(final_data[v])
        for k in final_data['year_month'].unique():
            c = final_data['year_month'] == k
            t = final_data[c]
            for i in t['FINAL_PRED_FLAG'].unique():
                for j in t['ACTUAL_DEMAND_FLAG'].unique():
                    c1 = t['FINAL_PRED_FLAG'] == i
                    c2 = t['ACTUAL_DEMAND_FLAG'] == j
                    # print(i,j)
                    # print((c1&c2).sum())
                    data_collect['Actual Sales'].append(j)
                    data_collect['Predicted Sales'].append(i)
                    data_collect['No of SKUS'].append((c1 & c2).sum())
        d = pd.DataFrame(data_collect)
        d['H1'] = filters["H1"]
        d['Channel'] = filters["Channel"]
        d['Total'] = filters["Total"]
        d['Statistical'] = filters["Statistical"]
        confusion_matrix_schema = pd.concat([confusion_matrix_schema, d], axis=0)

    return confusion_matrix_schema



In [116]:
limits=[10000,100000,200000]
cols="actual_demand"

In [117]:
# acc_final_data

In [119]:
final_data = get_data_from_rules(acc_final_data, "Statistical", 'confusion_matrix')
final_data

,year_month,sku,Channel,final_perdiction,actual_demand,SALES_SUM,Revision_Sales,mape,ACC_BUCKET,H1,one_unit_cost,actual_sales,Predicted
0,2023-12,1039,Export_Re-Export,6.620000e-24,0.0,0.0,0,NA,NA,Dairy,240.0,0.0,0.0
1,2024-01,1039,Export_Re-Export,6.620000e-24,0.0,0.0,0,NA,NA,Dairy,240.0,0.0,0.0
2,2024-02,1039,Export_Re-Export,6.620000e-24,0.0,0.0,0,NA,NA,Dairy,240.0,0.0,0.0
3,2024-03,1039,Export_Re-Export,6.620000e-24,0.0,0.0,0,NA,NA,Dairy,240.0,0.0,0.0
4,2024-04,1039,Export_Re-Export,6.620000e-24,0.0,0.0,0,NA,NA,Dairy,240.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
235729,2024-01,200852,Ecomm,0.000000e+00,0.0,0.0,0,nan,NA,Seafood,NaN,NaN,NaN
235730,2024-02,200852,Ecomm,0.000000e+00,0.0,0.0,0,nan,NA,Seafood,NaN,NaN,NaN
235731,2024-03,200852,Ecomm,0.000000e+00,0.0,0.0,0,nan,NA,Seafood,NaN,NaN,NaN
235732,2024-04,200852,Ecomm,0.000000e+00,0.0,0.0,0,nan,NA,Seafood,NaN,NaN,NaN


In [122]:
a= load_filters_(limits, cols, final_data)
pd.Series(a).unique()

array(['< SGD 10k', 'SGD 10K- 100K'], dtype=object)

In [104]:

def get_confusion_matrix_schema_(acc_final_data, data_mapping_accuracy_segmentation):
    confusion_matrix_schema = pd.DataFrame()
    for i in range(data_mapping_accuracy_segmentation.shape[0]):
        filters = data_mapping_accuracy_segmentation.iloc[i, :]
        columns_ = ["Statistical", "Total"]
        rules_ = filters.to_frame().loc[columns_, :]
        rules_index = rules_.index
        rules_values = rules_.iloc[:, 0].values
        rules_values = [rules_index[i] for i, j in enumerate(rules_values) if int(j) != 0]
        rules_values = '_'.join(rules_values)
        print(rules_values)
        final_data = get_data_from_rules(acc_final_data, rules_values, 'confusion_matrix')
        # display(final_data)
        limits = [10000, 100000, 200000]
        cols = "Predicted"
        final_data["FINAL_PRED_FLAG"] = load_filters_(limits, cols, final_data)
        cols = "actual_demand"
        final_data["ACTUAL_DEMAND_FLAG"] = load_filters_(limits, cols, final_data)
        for j in ['H1', "Channel"]:
            # print(j,filters[j])
            c = generate_condition(final_data, j, filters[j])
            final_data = final_data[c].reset_index(drop=True)
        if final_data.shape[0] == 0:
            continue
        data_collect = {
            "Actual Sales": [],
            "Predicted Sales": [],
            "No of SKUS": [],
            'year_month': []
        }
        # print(final_data['ACTUAL_DEMAND_FLAG'].unique())
        # v=final_data['ACTUAL_DEMAND_FLAG']=="0"
        # display(final_data[v])
        for k in final_data['year_month'].unique():
            c = final_data['year_month'] == k
            t = final_data[c]
            for i in t['FINAL_PRED_FLAG'].unique():
                for j in t['ACTUAL_DEMAND_FLAG'].unique():
                    c1 = t['FINAL_PRED_FLAG'] == i
                    c2 = t['ACTUAL_DEMAND_FLAG'] == j
                    # print(i,j)
                    # print((c1&c2).sum())
                    data_collect['Actual Sales'].append(j)
                    data_collect['Predicted Sales'].append(i)
                    data_collect['No of SKUS'].append((c1 & c2).sum())
                    data_collect['year_month'].append(k)
        d = pd.DataFrame(data_collect)
        d['H1'] = filters["H1"]
        d['Channel'] = filters["Channel"]
        d['Total'] = filters["Total"]
        d['Statistical'] = filters["Statistical"]
        confusion_matrix_schema = pd.concat([confusion_matrix_schema, d], axis=0)

    return confusion_matrix_schema


In [ ]:
 load_filters_(limits, cols, final_data)

In [121]:

def load_filters_(limits, cols, data):
    condition_list = []
    choice_list = []
    for j, i in enumerate(limits):
        if j == 0:
            # print( i)
            choice = f"< SGD {str(int(i/1000))}k"
            choice_list.append(choice)
            condi = data[cols] <= i
            condition_list.append(condi)
        else:
            # print(limits[j-1],i)
            choice = f"SGD {str(int(limits[j - 1]/1000))}K- {str(int(i/1000))}K"
            choice_list.append(choice)
            condi = (data[cols] > limits[j - 1]) & (data[cols] <= i)
            condition_list.append(condi)

        if j == len(limits) - 1:
            # print(i)
            choice = f"> SGD {str(int(i))}K"
            choice_list.append(choice)
            condi = (data[cols] > i)
            condition_list.append(condi)

    return np.select(condition_list, choice_list)

In [50]:
confusion_matrix_schema

,Actual Sales,Predicted Sales,No of SKUS,year_month,H1,Channel,Total,Statistical,client_id,last_updated_at
0,SGD 100K- 200K,SGD 100K- 200K,25,2023-12,Bakery,B2B,0,1,1,2024-08-18
1,SGD 10K- 100K,SGD 100K- 200K,16,2023-12,Bakery,B2B,0,1,1,2024-08-18
2,> SGD 200K,SGD 100K- 200K,14,2023-12,Bakery,B2B,0,1,1,2024-08-18
3,< SGD 10k,SGD 100K- 200K,9,2023-12,Bakery,B2B,0,1,1,2024-08-18
4,SGD 100K- 200K,SGD 10K- 100K,16,2023-12,Bakery,B2B,0,1,1,2024-08-18
...,...,...,...,...,...,...,...,...,...,...
91,> SGD 200K,> SGD 200K,874,2024-05,ALL,ALL,1,0,1,2024-08-18
92,< SGD 10k,SGD 10K- 100K,2070,2024-05,ALL,ALL,1,0,1,2024-08-18
93,SGD 10K- 100K,SGD 10K- 100K,2958,2024-05,ALL,ALL,1,0,1,2024-08-18
94,SGD 100K- 200K,SGD 10K- 100K,221,2024-05,ALL,ALL,1,0,1,2024-08-18


In [53]:
input_data_c.head(2)

,date,sku,year_month,Channel,TOTAL_QTY_BASEUOM_SUM,TOTAL_QTY_BASEUOM_COUNT,TOTAL_SALES_SUM,QTY_BASEUOM_SUM,QTY_BASEUOM_COUNT,SALES_SUM,KG_SUM,H1,last_year_demand,last_year_demand_non_contract,last_year_KG_SUM,one_unit_cost
0,2018-12-01,101.0,2018-12,B2B,NaN,NaN,NaN,0.0,NaN,0.0,NaN,Bakery,NaN,NaN,NaN,13.5
1,2019-01-01,101.0,2019-01,B2B,9.0,6.0,121.5,9.0,6.0,121.5,45.0,Bakery,NaN,NaN,NaN,13.5


In [55]:
forecast_history_data=pd.merge(forecast_history_data,input_data_c[['sku','year_month','Channel','QTY_BASEUOM_SUM']],on=['sku','year_month','Channel'],how='left')
forecast_history_data

,year_month,sku,Channel,final_perdiction,H1,QTY_BASEUOM_SUM
0,2023-12,1039,Export_Re-Export,6.620000e-24,Dairy,0.0
1,2024-01,1039,Export_Re-Export,6.620000e-24,Dairy,0.0
2,2024-02,1039,Export_Re-Export,6.620000e-24,Dairy,0.0
3,2024-03,1039,Export_Re-Export,6.620000e-24,Dairy,0.0
4,2024-04,1039,Export_Re-Export,6.620000e-24,Dairy,0.0
...,...,...,...,...,...,...
235729,2024-01,200852,Ecomm,0.000000e+00,Seafood,NaN
235730,2024-02,200852,Ecomm,0.000000e+00,Seafood,NaN
235731,2024-03,200852,Ecomm,0.000000e+00,Seafood,NaN
235732,2024-04,200852,Ecomm,0.000000e+00,Seafood,NaN


In [59]:
forecast_history_data.sample(20)

,year_month,sku,Channel,final_perdiction,H1,QTY_BASEUOM_SUM
28925,2024-05,68257,B2B,2.116667e+01,Bakery,17.0
110678,2024-02,53881,Retail,1.095788e+02,Non Food,93.0
107101,2024-01,108235,B2B,1.883333e+01,Meat,10.0
221241,2024-03,105486,B2B,1.500000e-16,PHD,NaN
147373,2024-01,58085,Retail,1.833333e+00,Non Food,0.0
83512,2024-04,108515,Export_Re-Export,6.000755e-01,Filling & Jam,0.0
126274,2024-04,67887,B2B,1.532375e+00,Non Food,0.0
176616,2023-12,52888,Ecomm,2.790000e-07,Non Food,NaN
226445,2024-05,101678,Retail,4.833333e+00,PHD,NaN
174600,2023-12,13679,Ecomm,3.333333e-01,Non Food,0.0


In [258]:
c=input_data_c['sku']==67008
c1=input_data_c['year_month']=='2024-04'
c3=input_data_c['Channel']=="B2B"
input_data_c[c & c1 & c3]

,date,sku,year_month,Channel,TOTAL_QTY_BASEUOM_SUM,TOTAL_QTY_BASEUOM_COUNT,TOTAL_SALES_SUM,QTY_BASEUOM_SUM,QTY_BASEUOM_COUNT,SALES_SUM,KG_SUM,H1,last_year_demand,last_year_demand_non_contract,last_year_KG_SUM,one_unit_cost
814293,2024-04-01,67008.0,2024-04,B2B,120.0,5.0,285.0,120.0,5.0,285.0,19.2,Non Food,80.0,80.0,12.8,2.4


In [254]:
data_['mape']=np.abs(data_['final_perdiction']-data_['QTY_BASEUOM_SUM'])/data_['QTY_BASEUOM_SUM']

In [276]:
data_=pd.merge(data_,input_data_c[['sku','year_month','SALES_SUM',"Channel"]],on=['sku','year_month','Channel'],how='left')

In [274]:
data_

,year_month,sku,Channel,final_perdiction,H1,one_unit_cost,QTY_BASEUOM_SUM,actual_sales,forecast_sales,mape,ACC_BUCKET
0,2023-12,1039,Export_Re-Export,6.620000e-24,Dairy,240.0,0.0,0.0,1.588800e-21,inf,NA
1,2024-01,1039,Export_Re-Export,6.620000e-24,Dairy,240.0,0.0,0.0,1.588800e-21,inf,NA
2,2024-02,1039,Export_Re-Export,6.620000e-24,Dairy,240.0,0.0,0.0,1.588800e-21,inf,NA
3,2024-03,1039,Export_Re-Export,6.620000e-24,Dairy,240.0,0.0,0.0,1.588800e-21,inf,NA
4,2024-04,1039,Export_Re-Export,6.620000e-24,Dairy,240.0,0.0,0.0,1.588800e-21,inf,NA
...,...,...,...,...,...,...,...,...,...,...,...
235729,2024-01,200852,Ecomm,0.000000e+00,Seafood,NaN,NaN,NaN,NaN,NaN,NA
235730,2024-02,200852,Ecomm,0.000000e+00,Seafood,NaN,NaN,NaN,NaN,NaN,NA
235731,2024-03,200852,Ecomm,0.000000e+00,Seafood,NaN,NaN,NaN,NaN,NaN,NA
235732,2024-04,200852,Ecomm,0.000000e+00,Seafood,NaN,NaN,NaN,NaN,NaN,NA


In [306]:
c1=data_['H1']=="Chocolates"
c2=data_['Channel']=="B2B"
c3=data_['year_month']=="2024-01"
data_[c1 & c3]

,year_month,sku,Channel,final_perdiction,H1,one_unit_cost,QTY_BASEUOM_SUM,actual_sales,forecast_sales,mape,ACC_BUCKET,SALES_SUM
71749,2024-01,213,B2B,4.853937e+02,Chocolates,15.574242,496.0,7724.824242,7.559639e+03,0.021384,HIGH,5676.95
71755,2024-01,295,B2B,4.455940e+01,Chocolates,136.000000,38.0,5168.000000,6.060079e+03,0.172616,HIGH,3116.00
71761,2024-01,2365,B2B,6.686505e+01,Chocolates,NaN,NaN,NaN,NaN,NaN,NA,NaN
71767,2024-01,9208,B2B,5.019477e+01,Chocolates,8.280000,48.0,397.440000,4.156127e+02,0.045724,HIGH,371.56
71773,2024-01,9779,B2B,1.271636e+02,Chocolates,13.000000,48.0,624.000000,1.653126e+03,1.649241,LOW,522.20
...,...,...,...,...,...,...,...,...,...,...,...,...
76303,2024-01,200502,Export_Re-Export,7.090000e-20,Chocolates,12.000000,0.0,0.000000,8.508000e-19,inf,NA,0.00
76309,2024-01,200670,Export_Re-Export,4.000000e+00,Chocolates,2.000000,24.0,48.000000,8.000000e+00,0.833333,LOW,48.00
76315,2024-01,200779,Export_Re-Export,2.700000e-15,Chocolates,16.220000,0.0,0.000000,4.379400e-14,inf,NA,0.00
76321,2024-01,200780,Export_Re-Export,2.350000e-09,Chocolates,5.900000,0.0,0.000000,1.386500e-08,inf,NA,0.00


In [308]:
data_[c1  & c3]['ACC_BUCKET'].value_counts()

ACC_BUCKET
NA        455
LOW       134
HIGH      107
MEDIUM     68
Name: count, dtype: int64

In [310]:
dd_=data_[c1  & c3].groupby(['ACC_BUCKET'],as_index=False)['SALES_SUM'].sum()

In [311]:
dd_['SALES_SUM']/dd_['SALES_SUM'].sum()

0    0.496911
1    0.214579
2    0.288510
3    0.000000
Name: SALES_SUM, dtype: float64

In [239]:
c=input_data_c['H1']=='Fruits'
c1=input_data_c['Channel']=="B2B"
input_data_c[c & c1]['sku'].nunique()

316

In [280]:
data_[c1 & c2 & c3]['ACC_BUCKET'].value_counts(normalize=True)

Series([], Name: proportion, dtype: float64)

In [264]:
data_[c1 & c2 & c3]['ACC_BUCKET'].value_counts()/(88+62+50)

ACC_BUCKET
NA        0.82
HIGH      0.44
LOW       0.31
MEDIUM    0.25
Name: count, dtype: float64

In [233]:
88+61+50

199

In [256]:
# data_['mape']=np.where(np.isinf(data_['mape']),"NA",data_['mape'])

In [288]:
c2=data_['mape']<=0.25
c3 = data_['mape']>0.25
c4= data_['mape']<=0.5
c5=data_['mape']>0.5
data_['ACC_BUCKET']=np.select([c2, c3 & c4,c5],['HIGH',"MEDIUM",'LOW'],default='NA')

In [289]:
data_['ACC_BUCKET']=np.where(np.isinf(data_['mape']),"NA",data_['ACC_BUCKET'])

In [290]:
data_['ACC_BUCKET'].value_counts(normalize=True)

ACC_BUCKET
NA        0.743457
LOW       0.122765
HIGH      0.075119
MEDIUM    0.058659
Name: proportion, dtype: float64

In [226]:
data_.to_clipboard(index=False)

In [313]:
name="MILAN"

In [356]:
c=input_data_c['sku']==53930
c1=input_data_c['Channel']=="B2B"
d=input_data_c[c & c1][['sku','year_month','QTY_BASEUOM_SUM']]

In [357]:
d.to_clipboard(index=False)

In [363]:
c=sku_master['Product']=='51093'
sku_master[c]["CrossPlantStatus"]

2525    DM
Name: CrossPlantStatus, dtype: object

In [374]:
c1=data_['H1']=="Bakery"
c2=data_['Channel']=="B2B"
c3=data_['year_month']=='2023-12'
data_[c1 & c2 & c3]['QTY_BASEUOM_SUM'].sum(),data_[c1 & c2 & c3]['final_perdiction'].astype(int).sum()

(146032.0, 148894)

In [375]:
(146032-148894)/149334

-0.01916509301297762

In [376]:
d=pd.read_clipboard()
d

,year_month,high_percentage_value,low_percentage_value,medium_percentage_value,others_percentage_value,high_sku_count,low_sku_count,medium_sku_count,others_sku_count,channel,h1,statistical_accuracy,total_accuracy,client_id,last_updated_at
0,2023-12-01,0.615452,0.179999,0.204549,0.000000,219.0,353.0,155.0,1546,B2B,Bakery,1,0,1,2024-08-20
1,2024-01-01,0.597503,0.119305,0.283191,0.000000,209.0,322.0,159.0,1583,B2B,Bakery,1,0,1,2024-08-20
2,2024-02-01,0.555682,0.191314,0.253004,0.000000,176.0,326.0,132.0,1639,B2B,Bakery,1,0,1,2024-08-20
3,2024-03-01,0.590536,0.160229,0.249234,0.000000,194.0,292.0,156.0,1631,B2B,Bakery,1,0,1,2024-08-20
4,2024-04-01,0.580182,0.174850,0.244967,0.000000,176.0,327.0,146.0,1624,B2B,Bakery,1,0,1,2024-08-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,2024-01-01,0.531668,0.239189,0.228371,0.000772,3094.0,4927.0,2358.0,28910,ALL,ALL,0,1,1,2024-08-20
956,2024-02-01,0.469943,0.260057,0.269341,0.000659,2767.0,4725.0,2092.0,29705,ALL,ALL,0,1,1,2024-08-20
957,2024-03-01,0.435231,0.259103,0.304790,0.000877,2802.0,4830.0,2352.0,29305,ALL,ALL,0,1,1,2024-08-20
958,2024-04-01,0.530277,0.223025,0.245649,0.001049,2936.0,4659.0,2262.0,29432,ALL,ALL,0,1,1,2024-08-20


In [379]:
acc_final_data_schema=d.copy()

In [380]:
acc_final_data_schema["high_percentage_value"]=np.round(acc_final_data_schema['high_percentage_value'],decimals=2)
acc_final_data_schema["medium_percentage_value"]=np.round(acc_final_data_schema['medium_percentage_value'],decimals=2)
acc_final_data_schema["low_percentage_value"]=np.round(acc_final_data_schema['low_percentage_value'],decimals=2)

In [382]:
acc_final_data_schema.to_clipboard(index=False)

In [387]:
d=pd.read_clipboard()
d

,A,B
0,1.4,5.0
1,NaN,5.0
2,4.2,5.0
3,5.0,5.0
4,NaN,33.0
5,NaN,2.0
6,7.0,NaN
7,8.0,55.0
8,999.0,565.0


In [391]:
np.round(d["A"])

0      1.0
1      NaN
2      4.0
3      5.0
4      NaN
5      NaN
6      7.0
7      8.0
8    999.0
Name: A, dtype: float64

In [393]:
d["A"].astype(int,errors='ignore')

0      1.4
1      NaN
2      4.2
3      5.0
4      NaN
5      NaN
6      7.0
7      8.0
8    999.0
Name: A, dtype: float64

"MILNA"


In [405]:
# pip install bash_kernel